In [1]:
import datetime
import fetcher
import compute
import pandas as pd
from finta import TA
import numpy as np

startDate = datetime.date(2020, 1, 1)
#endDate = datetime.date(2024, 7, 27)
endDate = datetime.date.today()
Stocks_list = []#"NVDA", "AAPL", "DKNG", "UBER"

# Retrieving tickers from a file
stocks = pd.read_csv("SCANS\\US 2024-08-09.csv")
#Neglecting tickers with Exchange values "AMEX" and "OTC"
stocks = stocks[~stocks["Exchange"].isin(["OTC", "AMEX"])]
#stocks = stocks[~stocks["Industry"].isin(["Biotechnology", "Pharmaceuticals: major", "Medical Specialties"])]
for ticker in stocks["Symbol"].tolist()[1:]:
    Stocks_list.append(ticker)

df_index = fetcher.__download_data( ["SPY"], startDate, endDate, "US")

info = {}
RS_scores = {}

RS_rating_test = {}

focus_list_IPO = pd.DataFrame(columns=['ticker','ADR','ADV'])
for ticker in Stocks_list:
    
    #Neglecting tickers with .A or .B or "/" in their names
    if len(ticker.split(".")) >1 or len(ticker.split("/")) >1:
        continue
    df = fetcher.__download_data([ticker], startDate, endDate, "US")
    
    # RS_Scores_qtrs ## Having this in the beginning is to have all the stocks while calculating percentile
    RS_score = compute.RS_score(df["Close"], df_index["Close"])
    RS_scores[ticker] = RS_score 
    
    # IPO stocks in the last 3 years
    if len(df) < 252*2:
        # Avg Volume*Price
        if len(df) > 30:
            ADV = compute.avg_dollar_vol(df, 30)
        
            """if ADV < 10000000:
                continue"""
    
        # ADR%
        if len(df) > 20:
            ADR = compute.ADR(df, 20)
            """if ADR < 2.25:
                continue"""
            new_row = {"ticker":ticker, "ADR": ADR, "ADV": ADV}
            focus_list_IPO = focus_list_IPO.append(new_row, ignore_index=True)
    else:
        # Avg Volume*Price
        ADV = compute.avg_dollar_vol(df, 30)

        # ADR%
        ADR = compute.ADR(df, 20)
        
        #52 week highs
        _52weekHigh = max(df["Close"][-252:])

        # RS
        #RS = compute.RS_line(df, df_index, 252)
        EMA10 = TA.EMA(df, period= 10)
        EMA21 = TA.EMA(df, period= 21)
        SMA50 =  np.convolve(df["Close"], np.ones(50)/50, mode='valid')
        SMA200 = np.convolve(df["Close"], np.ones(200)/200, mode='valid')
        slope50 = compute.linear_regression(SMA50[-5:])
        slope200 = compute.linear_regression(SMA200[-5:])
        #compute.calc_slope(SMA[-42:], 42)
        #print(angle)
        info[ticker] = {"Close": df["Close"][-1], "ADR" : ADR, "ADV" : ADV, "52weekHigh":_52weekHigh, "EMA10": EMA10[-1], "EMA21": EMA21[-1], "SMA50": SMA50[-1], "SMA200": SMA200[-1], "Slope50": slope50, "Slope200": slope200}
        #RS_rating_test[ticker] = compute.RS_rating_new(RS_score_qtrs)


# Sorting based on RS_score 
RS_scores = {ticker: RS_score for ticker, RS_score in sorted(RS_scores.items(), key=lambda item: item[1])}


# RS_rating for the stocks
RS_ratings = compute.RS_rating(RS_scores)

# Storing all info in a dataframe
focus = pd.DataFrame(columns=['ticker',"Close", 'ADR','ADV', '52weekHigh', 'RS_ratings', "EMA10", "EMA21","SMA50", "SMA200", "Slope50", "Slope200"])
for ticker, values in info.items():
    new_row = {"ticker":ticker, "Close" : values["Close"], "ADR": values["ADR"], "ADV": values["ADV"], '52weekHigh' : values["52weekHigh"], "RS_ratings": RS_ratings[ticker], "EMA10":values["EMA10"], "EMA21":values["EMA21"], "SMA50": values["SMA50"], "SMA200": values["SMA200"], "Slope50": values["Slope50"], "Slope200": values["Slope200"]}
    focus = focus.append(new_row, ignore_index=True)
    



reading from file:  yahoo_data/US/SPY2020-01-012024-08-09.csv
reading from file:  yahoo_data/US/WES2020-01-012024-08-09.csv
reading from file:  yahoo_data/US/TYRA2020-01-012024-08-09.csv
reading from file:  yahoo_data/US/EXPI2020-01-012024-08-09.csv
reading from file:  yahoo_data/US/DAC2020-01-012024-08-09.csv
reading from file:  yahoo_data/US/RILY2020-01-012024-08-09.csv
reading from file:  yahoo_data/US/ML2020-01-012024-08-09.csv
reading from file:  yahoo_data/US/TBLA2020-01-012024-08-09.csv
reading from file:  yahoo_data/US/SEZL2020-01-012024-08-09.csv
reading from file:  yahoo_data/US/PGEN2020-01-012024-08-09.csv
reading from file:  yahoo_data/US/GES2020-01-012024-08-09.csv
reading from file:  yahoo_data/US/CRON2020-01-012024-08-09.csv
reading from file:  yahoo_data/US/UDMY2020-01-012024-08-09.csv
reading from file:  yahoo_data/US/SVRA2020-01-012024-08-09.csv
reading from file:  yahoo_data/US/GRPN2020-01-012024-08-09.csv
reading from file:  yahoo_data/US/WAY2020-01-012024-08-09.csv

### Filtering IPO list

In [7]:
final_focus_IPO = focus_list_IPO.loc[(focus['ADR'] >=2.75) & (focus['ADV'] >=10000000)]


### Filtering NON IPO stocks

In [3]:
final_focus_NON_IPO = focus.loc[ (focus['ADR'] >=2) & (focus['ADV'] >=10000000) &(focus["Close"] >= focus['SMA200'])  & (focus['SMA50'] >=focus['SMA200']) & (focus['Close'] >=.8*focus['52weekHigh'])] #&& (focus["EMA10"] >= focus['SMA50']) & (focus["EMA21"] >= focus['SMA50'])  & (focus['RS_ratings'] >= 40) & (focus['Slope50'] >=0) & (focus['Slope200'] >=0)
len(final_focus_NON_IPO["ticker"].values)

850

### Sorting

In [36]:
final_focus_NON_IPO = pd.DataFrame(final_focus_NON_IPO.sort_values(by=['ADR'], ascending=False).to_numpy(), 
                   index=final_focus_NON_IPO.index, columns=final_focus_NON_IPO.columns)

In [50]:
stocks = stocks.replace({'/': '_'}, regex=True)

### Grouping by Sectors & Industries

In [37]:
sectors = {}
industries = {}
for index, row in final_focus_NON_IPO.iterrows():
    sector = stocks.loc[stocks['Symbol'] == row["ticker"]]["Sector"].values[0]
    #print(sector)
    industry = stocks.loc[stocks['Symbol'] == row["ticker"]]["Industry"].values[0]
    #print(industry)
    if sector not in sectors.keys():
        sectors[sector] = {}
        
    if industry not in sectors[sector].keys():
        #print(industry)
        sectors[sector][industry] = []
    
    #print(sectors)
            #print(sectors[sector])
    sectors[sector][industry].append(row["ticker"])


In [38]:
print(sectors.keys())

dict_keys(['Communications', 'Retail trade', 'Producer manufacturing', 'Technology services', 'Health technology', 'Health services', 'Electronic technology', 'Finance', 'Industrial services', 'Consumer durables', 'Consumer services', 'Transportation', 'Consumer non-durables', 'Non-energy minerals', 'Distribution services', 'Energy minerals', 'Process industries', 'Commercial services', 'Utilities', 'Miscellaneous'])


In [39]:
sectors

{'Communications': {'Major telecommunications': ['LUMN', 'FYBR', 'LILAK'],
  'Specialty telecommunications': ['ASTS', 'SATS', 'LBTYK', 'LBTYA']},
 'Retail trade': {'Specialty stores': ['EVGO',
   'CVNA',
   'SAH',
   'RUSHA',
   'GPI',
   'KMX',
   'MUSA',
   'ORLY',
   'CASY'],
  'Apparel/Footwear retail': ['CAL', 'BURL'],
  'Internet retail': ['MELI', 'EBAY'],
  'Food retail': ['SFM'],
  'Department stores': ['OLLI'],
  'Home improvement chains': ['LOW', 'HD']},
 'Producer manufacturing': {'Electrical products': ['EOSE', 'BDC'],
  'Miscellaneous manufacturing': ['DAKT', 'AWI'],
  'Building products': ['SPXC', 'GFF', 'MHK', 'ALLE', 'MAS'],
  'Industrial machinery': ['OII', 'AAON', 'CECO', 'LII', 'PNR', 'MWA'],
  'Metal fabrication': ['MLI', 'VMI'],
  'Trucks/Construction/Farm machinery': ['TRN', 'FSS'],
  'Office equipment/Supplies': ['MLKN', 'HNI'],
  'Auto parts: OEM': ['CMI', 'ALSN']},
 'Technology services': {'Packaged software': ['AUR',
   'COMP',
   'ZETA',
   'IOT',
   'PLTR',


In [54]:
a = []
for industry in sectors.values():
    for ticker in list(industry.values())[0]:
        a.append(ticker)

In [56]:
print(a)

['LUMN', 'FYBR', 'LILAK', 'EVGO', 'CVNA', 'SAH', 'RUSHA', 'GPI', 'KMX', 'MUSA', 'ORLY', 'CASY', 'EOSE', 'BDC', 'AUR', 'COMP', 'ZETA', 'IOT', 'PLTR', 'AGYS', 'CVLT', 'ALKT', 'CWAN', 'PLUS', 'MANH', 'ACIW', 'NOW', 'FICO', 'DFIN', 'VERX', 'SMAR', 'IDCC', 'EA', 'CDNA', 'TMDX', 'LNTH', 'NEOG', 'PODD', 'ICUI', 'MYGN', 'RMD', 'RVTY', 'MMM', 'MMSI', 'ISRG', 'STE', 'TMO', 'DHR', 'GMED', 'WGS', 'GH', 'ENSG', 'ADUS', 'RUN', 'TRUP', 'BWIN', 'CNO', 'CINF', 'GNW', 'PGR', 'PCT', 'CWST', 'CLH', 'GFL', 'HOV', 'MHO', 'CCS', 'GRBK', 'MTH', 'KBH', 'PATK', 'TOL', 'FTDR', 'TPH', 'DHI', 'TMHC', 'PHM', 'LEN', 'FIP', 'PRDO', 'PLNT', 'LRN', 'ATGE', 'UNF', 'SCI', 'XPO', 'SNDR', 'TFII', 'HBI', 'WWW', 'CRS', 'ATI', 'BCC', 'HLMN', 'DNOW', 'AIT', 'GWW', 'CLB', 'SMG', 'CRTO', 'WTTR', 'CWT', 'TPL']


### Industries in each sector

In [5]:
a = stocks['Industry'].unique()
#for index, row in stocks.iterrows():
sectors = {}
for industry in a:
    sector = stocks.loc[stocks['Industry'] == industry]["Sector"].values[0]
    if sector not in sectors:
        sectors[sector] = []
    sectors[sector].append(industry)

In [6]:
sectors

{'Utilities': ['Gas distributors',
  'Alternative power generation',
  'Water utilities',
  'Electric utilities'],
 'Industrial services': ['Oil & gas pipelines',
  'Oilfield services/Equipment',
  'Environmental services',
  'Engineering & construction',
  'Contract drilling'],
 'Health technology': ['Biotechnology',
  'Pharmaceuticals: other',
  'Pharmaceuticals: major',
  'Medical specialties',
  'Pharmaceuticals: generic'],
 'Finance': ['Real estate development',
  'Investment banks/Brokers',
  'Investment managers',
  'Real estate investment trusts',
  'Insurance brokers/Services',
  'Regional banks',
  'Property/Casualty insurance',
  'Finance/Rental/Leasing',
  'Financial conglomerates',
  'Major banks',
  'Multi-line insurance',
  'Savings banks',
  'Life/Health insurance',
  'Specialty insurance'],
 'Transportation': ['Marine shipping',
  'Trucking',
  'Air freight/Couriers',
  'Other transportation',
  'Airlines',
  'Railroads'],
 'Technology services': ['Packaged software',
